In [1]:
#Importing the necessary package
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
from datetime import datetime
import numpy as np
import pyodbc

# Creating connection with the database
conn = pyodbc.connect(
    "Driver={SQL Server Native Client 11.0};"
    "Server=DESKTOP-AV9L7C3;"
    "Database=Amazon;"
    "Trusted_Connection=yes"
)

cursor = conn.cursor()

In [2]:
#Getting the page information
url = 'https://www.amazon.com.br/gp/bestsellers/books/'

r = requests.get(url)
soup = bs(r.content, 'html.parser')

In [3]:
#Analysing if we have all products from specific page
results = soup.find_all('div', {'class': 'a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc'})
len(results)

50

In [4]:
# Creating a function to extract the informations product
def extract_information(product):
    position = product.find('span', {'class': 'zg-bdg-text'}).text.strip('#')
    title = product.find('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'}).text
    author = product.find('div', {'class': 'a-row a-size-small'}).text
    
    try:
        stars = product.find('span', {'class': 'a-icon-alt'}).text.split()[0].replace(',','.')
    except AttributeError:
        stars=0

    try:
        reviews = product.find(lambda tag: tag.name =='span' and tag.get('class') == ['a-size-small']).text.replace('.','')
    except AttributeError:
        reviews = 0    

    results = [position, title, author, stars, reviews]

    return results


In [5]:
#Creating a dataframe with information from each product
product_infos = []

for result in results:
        product_infos.append(extract_information(result))

df = pd.DataFrame(product_infos, columns=['Position', 'Title', 'Author', 'Stars', 'Reviews'])

In [6]:
# Changing column type
df = df.astype({'Position': 'int',
           'Stars': 'float',
           'Reviews': 'int'})

df.head()

,Position,Title,Author,Stars,Reviews
0,1,Mentirosos – Sucesso no TikTok,E. Lockhart,4.5,18130
1,2,É assim que começa (Vol. 2 É assim que acaba),Colleen Hoover,4.8,34355
2,3,Da Silva: a Grande Fake News da Esquerda: o Pe...,Pavinatto,4.8,401
3,4,Hamnet: Edição especial com brindes e revista ...,Maggie O'Farrell,4.7,510
4,5,Goseki Kojima,Goseki Kojima,0.0,0


In [7]:
# The last part is to insert the values inside our database, and then we can analyze and make decisions.
sql = '''
INSERT INTO ProductsInformation (Position, Title, Author, Stars, Reviews) VALUES (?, ?, ?, ?, ?)
'''

cursor.fast_executemany = True

cursor.executemany(sql, df.values.tolist())

conn.commit()